# Reload changes from files

In [1]:
%load_ext autoreload
%autoreload 2

# Import from files

In [12]:
import numpy as np
import matplotlib.pyplot as plt


from BFS import BFS
from GenerateGrid import generate_grid
from Time import now, elapsed

# Make random grid and positions

In [3]:
def show_pos_in_grid(grid, start_pos, goal_pos):
    display_grid = np.array(grid, copy=True)
    x, y, _ = start_pos
    display_grid[y][x] = 8  # Mark the start position with an 8
    x, y = goal_pos
    display_grid[y][x] = 9  # Mark the goal position with a 9
    return display_grid

grid, start_pos, goal_pos = generate_grid(size=10, wall_num=10)
print(grid)
print("Start position:", start_pos)
print("Goal position:", goal_pos)
print(show_pos_in_grid(grid, start_pos, goal_pos))

"""
wall_collision_directions = [
    (0, -1),  # North
    (0, 0),   # Center
    (-1, -1), # North-West
    (-1, 0)   # West
]
"""

Checking position (4, 3)
Checking walls at positions: [(4, 2), (4, 3), (3, 2), (3, 3)]
Checking position (2, 2)
Checking walls at positions: [(2, 1), (2, 2), (1, 1), (1, 2)]
Checking position (7, 0)
Checking walls at positions: [(7, -1), (7, 0), (6, -1), (6, 0)]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [1 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Start position: (2, 2, 1)
Goal position: (7, 0)
[[0 0 0 0 0 0 0 9 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 8 0 0 0 0 0 1 0]
 [1 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


'\nwall_collision_directions = [\n    (0, -1),  # North\n    (0, 0),   # Center\n    (-1, -1), # North-West\n    (-1, 0)   # West\n]\n'

# Test BFS

In [4]:
"""
advance_directions = {
    0: (0, -1),  # North
    1: (1, 0),   # East
    2: (0, 1),   # South
    3: (-1, 0)   # West
}
"""

result = BFS(grid, start_pos, goal_pos)
if result:
    path = result
    print("Path found:", path[1:])
    print("Number of explored nodes:", path[0])
else:
    print("No path found")

Path found: ['a2', 'a3', 'G', 'a2']
Number of explored nodes: 4


# Generate test grids

In [5]:
def generate_test_grid(size=10, wall_num=1, n=10):
    """
    Generate multiple test grids with specified size and wall percentage.
    Args:
        size (int): Size of the grid (size x size).
        wall_num (int): Number of walls in the grid. Is multiplied by size to get the number of walls.
        n (int): Number of grids to generate.
    """
    grids = []
    for _ in range(n):
        grid, start_pos, goal_pos = generate_grid(size=size, wall_num=int(size*wall_num))
        grids.append((grid, start_pos, goal_pos))
    return grids


def test_BFS_on_multiple_grids(size=10, wall_num=1, n=10):
    test_grids = generate_test_grid(size=size, wall_num=wall_num, n=n)
    results = []
    for grid, start_pos, goal_pos in test_grids:
        start = now()
        result = BFS(grid, start_pos, goal_pos)
        results.append((elapsed(start), result))
    return results



# Test the test functions

In [11]:
for i in range(10, 51, 10):
    results = test_BFS_on_multiple_grids(size=i, wall_num=1, n=10)
    avg_time = sum([res[0] for res in results]) / len(results)

    #print(f"Results: {results}")
    print(f"Average time for BFS on {len(results)} grids of size {i}x{i} with {i} walls: {avg_time:.6f} seconds.")


Average time for BFS on 10 grids of size 10x10 with 10 walls: 0.002042 seconds.
Average time for BFS on 10 grids of size 20x20 with 20 walls: 0.017004 seconds.
Average time for BFS on 10 grids of size 30x30 with 30 walls: 0.027998 seconds.
Average time for BFS on 10 grids of size 40x40 with 40 walls: 0.078094 seconds.
Average time for BFS on 10 grids of size 50x50 with 50 walls: 0.117675 seconds.


# Mettre les resultats dans un plot

In [ ]:
def plot_performance():

    sizes = list(range(10, 51, 10))
    times = []

    for i in sizes:
        results = test_BFS_on_multiple_grids(size=i, wall_num=1, n=10)
        avg_time = sum([res[0] for res in results]) / len(results)
        times.append(avg_time)

    plt.plot(sizes, times, marker='o')
    plt.title('BFS Performance on Different Grid Sizes')
    plt.xlabel('Grid Size (N x N)')
    plt.ylabel('Average Time (seconds)')
    plt.grid(True)
    plt.show()

plot_performance()